In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain_groq import ChatGroq

In [2]:
import csv

csv_file_path = "SystemReq_EPS.csv"

def load_csv(file_path):
    requirements = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            requirements.append(row)
    return requirements

In [3]:
requirements_doc = load_csv(csv_file_path)

from langchain_core.documents import Document

documents = [(Document(page_content=str( {
            "Primary Text": item['Primary Text'],
            "Verification_Criteria": item['Verification_Criteria'],
        }), metadata={"Requirement ID": item["id"]})) for item in requirements_doc]

In [4]:
import uuid
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

ids = [str(uuid.uuid4()) for _ in range(len(documents))] 

unique_collection_name = f"collection_{uuid.uuid4()}"

client = Chroma(collection_name=unique_collection_name, embedding_function=embeddings_model)

# Add documents
client.add_documents(documents=documents, ids = ids)

['058d00ea-3e9a-4d4a-a5f1-da03e395f5aa',
 '90dfda65-6b20-4410-a02e-fd551ba417c4',
 'ca0b8329-3ff7-4dfe-b4a9-092c0df4dcb7',
 '0f2c99d9-9ab8-4d80-bfb7-33e9d8876341',
 'ba6949d6-73fb-4b75-bd40-0868eb593dd8',
 'c1c56928-1691-489b-b363-fc738257662d',
 '141f6334-efe4-49f6-827f-f41ff2872494',
 'b3cbc089-4ba5-467b-ba6c-7767a9ec53f6',
 '28910fe9-2ab4-498b-a63d-ac420235229f',
 'a18db2df-0284-453e-9f0a-c8e3d47cf1d2',
 'fc08e822-7d10-4c90-a6ec-c2bda4ebe11c',
 'ae3f7c7c-f1f6-4bf4-9f32-33467894146f',
 'e5248c76-c54c-4b1c-aa75-caddce455ee3',
 '23daac4f-0d27-4b0b-9239-9a39092bb4fe',
 '4aaa7194-b5db-424e-9a44-70a43758c71e',
 '15da607c-338a-4a13-9ece-043fed76991b',
 '44fc4a9e-7f2a-4150-8f62-144c4668867e',
 '1700392d-1937-4bad-8ea1-b480752a9ea5',
 'c2f05475-20fc-4fd4-b59a-10dc89c1169c']

In [8]:
import re

def extract_xml(text: str, tag: str) -> str:
    """
    Extracts the content of the specified XML tag from the given text. Used for parsing structured responses 

    Args:
        text (str): The text containing the XML.
        tag (str): The XML tag to extract content from.

    Returns:
        str: The content of the specified XML tag, or an empty string if the tag is not found.
    """
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""

# retrieving the relevant documents from vector store
def retrieve_documents(query_text, collection, top_k=4):
    """Fetching relevant document from vector store """
        
    documents=[]
    retrieve_doc = collection.similarity_search(query_text, k=top_k)

    for item in retrieve_doc:
         documents.append({
                "content": item.page_content,
                "source": item.metadata,
            })  
    
    return documents

#cleanising the retrieved vector store document
def clean_text(relevant_document):
    content_list = []
    for item in relevant_document:
        # Clean the 'content' field
        content = item['content']
        content = content.replace("'", '"')
        content = re.sub(r'[\'\-•]""', "", content)  # Remove specified characters
        content = re.sub(r'\s{2,}', ' ', content).strip()  # Normalize whitespace and trim
        item['content'] = content

        # Replace data in 'Verification_Criteria' field if it exists
        #for var in item:
        # criteria = item['Verification_Criteria']  # Update cleaned criteria in the item
        
        # combined_content = f"Primary Text: {content}\nVerification Criteria: {criteria}\n\n"
        content_list.append(content)
        cleanised_document = ''.join(content_list)
    return cleanised_document

def llm_call(prompt: str, system_prompt: str = "", model="llama3-70b-8192") -> str:
    """
    Calls the model with the given prompt and returns the response.

    Args:
        prompt (str): The user prompt to send to the model.
        system_prompt (str, optional): The system prompt to send to the model. Defaults to "".
        model (str, optional): The model to use for the call. Defaults to "llama3-70b-8192".

    Returns:
        str: The response from the language model.
    """
    
    messages = [{"role": "user", "content": prompt}]
    
    llm = ChatGroq(
    model=model,
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY")
    )

    response = llm.invoke(messages)
    return response.content.strip()

In [9]:
ORCHESTRATOR_PROMPT = """ You are provided with requirements of Automotive Electronic Control Unit(ECU). 
Analyze the requirement and understand the context of the it based on documents provided. You understand the requirement completely if you have any doubt please ask for clarifications
        requirement: {requirement}
        documents: {documents}

Return your response in this format:

<analysis>
Explain your understanding of the given requirement in detail. Break down it in detail.
</analysis>

<tests>
Based on the analysis, explain how the respective requirement can be fully validated / tested. Provide the response in below format:
    Test Number: (1,2,3 so on...)
    Objective: Objective of the test.
    Test Condition: How to perform the Test.
    Expected outcome: Expected outcome after performing the test.
    Validation: How to valid that the ECU behaves as per requirement.
</tests>

"""

In [10]:
WORKER_PROMPT = """ You are a test case generation assistant who will create test cases for given requirement. You are well aware of software testing and its testing techniques.
You should understand the requirement and test condition provided, then accordingly generate test cases for given specific Test conditions.

Requirements context:{context_requirement}
Test Conditions: {test_conditions} 

The test conditons provided to you will be in below format:
    Test (1,2,3 so on...): Scope of the test
    Objective: Objective of the test.
    Test Condition: How to perform the Test.
    Expected outcome: Expected outcome after performing the test.
    Validation: How to valid that the ECU behaves as per requirement.

The test cases should be structured, clear, and cover all possible validation scenarios for each condition. Make your designing test cases with different test techniques. 

Return your response in this format, with no preamble:

<test_cases>
Test Case ID: Unique ID (TC1, TC2, TC3 so on)
Objective: Objective of test case.
Test Design Technique: Test design technique used for this test
Pre-conditions: What are the preconditions before performing the test.
Steps to Execute: Detailed test steps
Expected Results: Expected outcome for respective Execution step 
</test_cases>

"""

In [11]:
from typing import Dict, List, Optional
    
class Orchestrator:
    """Break down tasks and run them in parallel using worker LLMs."""
    
    def __init__(self,orchestrator_prompt: str, worker_prompt:str):
        self.orchestrator_prompt = orchestrator_prompt
        self.worker_prompt = worker_prompt
    
    # formating prompt by replacing {} variables with its actual value 
    def _format_prompt(self, template: str,  **kwargs) -> str:
        """Format a prompt template with variables."""
        try:
            return template.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Missing required prompt variable: {e}")

    def process_llm(self, requirement: str, client: object) -> Dict:
        """Orchestrate tasks using the LLM based on the given requirement.
        
        Args:
            requirement (str): The requirement to process.
        
        Returns:
            str: Output from the orchestrator.
        """
        unpack = {}
        
        # retrieving and cleansing the document
        raw_documents = retrieve_documents(requirement,client)
        documents = clean_text(raw_documents)
    
        orchestrator_input = self._format_prompt(
            self.orchestrator_prompt,
            requirement=requirement,
            documents=documents,
            **unpack
        )
        orchestrator_response = llm_call(orchestrator_input)
        
        analysis = extract_xml(orchestrator_response, "analysis")
        test_scenarios = extract_xml(orchestrator_response, "tests")
        
        print("\n=== ORCHESTRATOR OUTPUT ===")
        print(f"\nANALYSIS:\n{analysis}")
        print(f"\nTEST SCENARIOS:\n{test_scenarios}")

        # Step 2: Process each task
        worker_input = self._format_prompt(
                self.worker_prompt,
                context_requirement=analysis,
                test_conditions=test_scenarios,
                **unpack
        )
            
        worker_response = llm_call(worker_input)
        #print(worker_response)
        TC_result = extract_xml(worker_response, "test_cases")
            
        print(f"\n=== WORKER RESULT ===\n{TC_result}\n")
        
        return [analysis, TC_result]

In [ ]:
requirements_doc

In [12]:
orchestrator = Orchestrator(
    orchestrator_prompt=ORCHESTRATOR_PROMPT,
    worker_prompt=WORKER_PROMPT,
)
test_cases_dict = {}
final_result = {}
# Iterate through the requirements document
for item in requirements_doc:
    if item['Artifact Type'] == 'SYS Requirement':
        print(f"Generating test case for Requirement ID: {item['id']}")

        # Process the requirement using the orchestrator
        result = orchestrator.process_llm(requirement=item['Primary Text'], client=client)

        # Prepare the test case dictionary
        test_cases_dict = {
            'Requirement ID': item['id'],
            'Analysis': result[0],
            'Test Cases': result[1]
        }

        # Add the test case dictionary to the final result
        final_result.setdefault(item['id'], []).append(test_cases_dict)

Generating test case for Requirement ID: 245103

=== ORCHESTRATOR OUTPUT ===

ANALYSIS:


The given requirement is for an Automotive Electronic Control Unit (ECU) that controls the steering system of a vehicle. The ECU takes input from the steering wheel angle, steering torque, and vehicle speed to provide assistive torque to the driver and adjust the steering angle at the wheels.

The requirement can be broken down into three main sections:

1. **Steering Wheel Angle**: The ECU shall process the steering wheel angle input, which can range from ±450° (effective within ±180°), and adjust the steering angle at the wheels by 8° for every 90° of steering wheel input. The ECU shall also provide assistive torque ranging from 1Nm to 15Nm based on the steering wheel angle and vehicle speed.

2. **Steering Torque**: The ECU shall respond to the torque applied by the driver to the steering wheel, which can range from 0-15Nm, and generate assistive torque up to a 4:1 assist ratio. The ECU shall a

In [ ]:
final_result['245107']